In [53]:
import pandas as pd
import psycopg2 as pg2
import psycopg2.extras as extras

# Load and explore Excel table

In [268]:
data = pd.read_excel(
    'Sample - Superstore.xlsx', 
    index_col='Row ID', 
    parse_dates=['Order Date', 'Ship Date'], 
    dtype={'Postal Code': object}
)
data.index.name = data.index.name.lower().replace(' ', '_')
data.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in data.columns]

C:\Users\space\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [155]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 1 to 9994
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       9994 non-null   object        
 1   order_date     9994 non-null   datetime64[ns]
 2   ship_date      9994 non-null   datetime64[ns]
 3   ship_mode      9994 non-null   object        
 4   customer_id    9994 non-null   object        
 5   customer_name  9994 non-null   object        
 6   segment        9994 non-null   object        
 7   country        9994 non-null   object        
 8   city           9994 non-null   object        
 9   state          9994 non-null   object        
 10  postal_code    9983 non-null   object        
 11  region         9994 non-null   object        
 12  product_id     9994 non-null   object        
 13  category       9994 non-null   object        
 14  sub_category   9994 non-null   object        
 15  product_name   9994 non-nu

,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,person,returned
row id,,,,,,,,,,,,,,,,,,,,,
1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,Cassandra Brandow,No
2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,Cassandra Brandow,No
3,CA-2018-138688,2018-06-12,2018-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,Anna Andreadi,No
4,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,Cassandra Brandow,No
5,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,Cassandra Brandow,No


In [156]:
data[data['postal_code'].isna()][['state', 'city']].drop_duplicates()

,state,city
row id,,
2235,Vermont,Burlington


In [298]:
data['postal_code'] = data['postal_code'].fillna('05401')

# Connection to PostgreSQL and insertion data to database

In [199]:
conn = psycopg2.connect("dbname='de101_module2' user=postgres password=1315420")

In [192]:
def insert_into_table(table:str, data:pd.DataFrame, conn:pg2.connect):
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, ','.join(data.columns))
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, data.values)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return
    print("the dataframe is inserted")
    cursor.close()

In [250]:
def select_from_table(table:str, columns:list, conn:pg2.connect):
    query = "SELECT %s FROM %s" % (','.join(columns), table)
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return pd.DataFrame(cursor.fetchall(), columns=columns)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return
    cursor.close()

### Customers

In [172]:
customers = data[['customer_id', 'customer_name', 'segment']].drop_duplicates().reset_index(drop=True)
customers.head()

,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer


In [173]:
insert_into_table('dw.customers', customers, conn)

the dataframe is inserted


### Regions

In [180]:
regions = data[['region', 'person']].drop_duplicates().reset_index(drop=True)
regions

,region,person
0,South,Cassandra Brandow
1,West,Anna Andreadi
2,Central,Kelly Williams
3,East,Chuck Magee


In [181]:
insert_into_table('dw.regions', regions, conn)

the dataframe is inserted


### Geography

In [182]:
geography = data[['country', 'city', 'state', 'postal_code', 'region']].drop_duplicates().reset_index(drop=True)
print("Rows count: ", len(geography))
geography.head()

Rows count:  632


,country,city,state,postal_code,region
0,United States,Henderson,Kentucky,42420,South
1,United States,Los Angeles,California,90036,West
2,United States,Fort Lauderdale,Florida,33311,South
3,United States,Los Angeles,California,90032,West
4,United States,Concord,North Carolina,28027,South


In [183]:
regions_uid = regions.reset_index(names='region_uid')[['region_uid', 'region']]
regions_uid[['region_uid']] = regions_uid[['region_uid']] + 1 
regions_uid

,region_uid,region
0,1,South
1,2,West
2,3,Central
3,4,East


In [184]:
geography = geography.merge(regions_uid, on='region')
geography = geography[geography.columns.drop('region')]
geography.head()

,country,city,state,postal_code,region_uid
0,United States,Henderson,Kentucky,42420,1
1,United States,Fort Lauderdale,Florida,33311,1
2,United States,Concord,North Carolina,28027,1
3,United States,Melbourne,Florida,32935,1
4,United States,Springfield,Virginia,22153,1


In [185]:
insert_into_table('dw.geography', geography, conn)

the dataframe is inserted


### Products

In [186]:
products = data[['product_id', 'product_name', 'category', 'sub_category']].drop_duplicates()
print("Rows count: ", len(products))
products.head()

Rows count:  1894


,product_id,product_name,category,sub_category
row id,,,,
1,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
2,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
3,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
4,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
5,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage


In [188]:
insert_into_table('dw.products', products, conn)

the dataframe is inserted


### Orders

In [270]:
orders = data[['order_id', 'order_date', 'customer_id', 'product_id', 'product_name']].reset_index()
print("Rows count: ", len(orders))
orders.head()

Rows count:  9994


,row_id,order_id,order_date,customer_id,product_id,product_name
0,1,CA-2018-152156,2018-11-08,CG-12520,FUR-BO-10001798,Bush Somerset Collection Bookcase
1,2,CA-2018-152156,2018-11-08,CG-12520,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,3,CA-2018-138688,2018-06-12,DV-13045,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...
3,4,US-2017-108966,2017-10-11,SO-20335,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table
4,5,US-2017-108966,2017-10-11,SO-20335,OFF-ST-10000760,Eldon Fold 'N Roll Cart System


In [264]:
prod_df = select_from_table('dw.products', ['product_uid', 'product_id', 'product_name'], conn)
cust_df = select_from_table('dw.customers', ['customer_uid', 'customer_id'], conn)

In [271]:
orders = orders.merge(prod_df, on=['product_id', 'product_name'], how='left')
orders = orders.merge(customers_ids, on='customer_id', how='left')
orders = orders[orders.columns.drop(['product_id', 'product_name', 'customer_id'])]
print("Rows count: ", len(orders))
orders.head()

Rows count:  9994


,row_id,order_id,order_date,product_uid,customer_uid
0,1,CA-2018-152156,2018-11-08,100000,10000
1,2,CA-2018-152156,2018-11-08,100001,10000
2,3,CA-2018-138688,2018-06-12,100002,10001
3,4,US-2017-108966,2017-10-11,100003,10002
4,5,US-2017-108966,2017-10-11,100004,10002


In [272]:
insert_into_table('dw.orders', orders, conn)

the dataframe is inserted


### Shipping

In [402]:
shipping = data[['order_id', 'order_date', 'ship_date', 'ship_mode', 'state', 'city', 'postal_code']].drop_duplicates()
print("Rows count: ", len(shipping))
shipping.tail()

Rows count:  5009


,order_id,order_date,ship_date,ship_mode,state,city,postal_code
row_id,,,,,,,
9987,CA-2018-125794,2018-09-29,2018-10-03,Standard Class,California,Los Angeles,90008
9988,CA-2019-163629,2019-11-17,2019-11-21,Standard Class,Georgia,Athens,30605
9990,CA-2016-110422,2016-01-21,2016-01-23,Second Class,Florida,Miami,33180
9991,CA-2019-121258,2019-02-26,2019-03-03,Standard Class,California,Costa Mesa,92627
9994,CA-2019-119914,2019-05-04,2019-05-09,Second Class,California,Westminster,92683


In [403]:
ord_df = select_from_table('dw.orders', ['order_uid', 'order_id', 'order_date'], conn)
ord_df['order_date'] = ord_df['order_date'].astype('datetime64[ns]')

geo_df = select_from_table('dw.geography', ['geo_id', 'state', 'city', 'postal_code'], conn)
geo_df['postal_code'] = geo_df['postal_code'].astype(str)

shipping['postal_code'] = shipping['postal_code'].astype(str)

In [404]:
shipping = shipping.merge(ord_df, on=['order_id', 'order_date'], how='left')
shipping = shipping.merge(geo_df, on=['state', 'city', 'postal_code'], how='left')
shipping = shipping[shipping.columns.drop(['order_date', 'state', 'city', 'postal_code'])]
print("Rows count: ", len(shipping))
shipping.head()

Rows count:  9994


,order_id,ship_date,ship_mode,order_uid,geo_id
0,CA-2018-152156,2018-11-11,Second Class,1000000,1000
1,CA-2018-152156,2018-11-11,Second Class,1000001,1000
2,CA-2018-138688,2018-06-16,Second Class,1000002,1136
3,US-2017-108966,2017-10-18,Standard Class,1000003,1001
4,US-2017-108966,2017-10-18,Standard Class,1000004,1001


In [405]:
insert_into_table('dw.shipping', shipping, conn)

the dataframe is inserted


### Returns